# Project 2: Web Traffic Analysis
**This is the second of three mandatory projects to be handed in as part of the assessment for the course 02807 Computational Tools for Data Science at Technical University of Denmark, autumn 2019.**

#### Practical info
- **The project is to be done in groups of at most 3 students**
- **Each group has to hand in _one_ Jupyter notebook (this notebook) with their solution**
- **The hand-in of the notebook is due 2019-11-10, 23:59 on DTU Inside**

#### Your solution
- **Your solution should be in Python**
- **For each question you may use as many cells for your solution as you like**
- **You should document your solution and explain the choices you've made (for example by using multiple cells and use Markdown to assist the reader of the notebook)**
- **You should not remove the problem statements**
- **Your notebook should be runnable, i.e., clicking [>>] in Jupyter should generate the result that you want to be assessed**
- **You are not expected to use machine learning to solve any of the exercises**
- **You will be assessed according to correctness and readability of your code, choice of solution, choice of tools and libraries, and documentation of your solution**

## Introduction
In this project your task is to analyze a stream of log entries. A log entry consists of an [IP address](https://en.wikipedia.org/wiki/IP_address) and a [domain name](https://en.wikipedia.org/wiki/Domain_name). For example, a log line may look as follows:

`192.168.0.1 somedomain.dk`

One log line is the result of the event that the domain name was visited by someone having the corresponding IP address. Your task is to analyze the traffic on a number of domains. Counting the number of unique IPs seen on a domain doesn't correspond to the exact number of unique visitors, but it is a good estimate.

Specifically, you should answer the following questions from the stream of log entries.

- How many unique IPs are there in the stream?
- How many unique IPs are there for each domain?
- How many times was IP X seen on domain Y? (for some X and Y provided at run time)

**The answers to these questions can be approximate!**

You should also try to answer one or more of the following, more advanced, questions. The answers to these should also be approximate.

- How many unique IPs are there for the domains $d_1, d_2, \ldots$?
- How many times was IP X seen on domains $d_1, d_2, \ldots$?
- What are the X most frequent IPs in the stream?

You should use algorithms and data structures that you've learned about in the lectures, and you should provide your own implementations of these.

Furthermore, you are expected to:

- Document the accuracy of your answers when using algorithms that give approximate answers
- Argue why you are using certain parameters for your data structures

This notebook is in three parts. In the first part you are given an example of how to read from the stream (which for the purpose of this project is a remote file). In the second part you should implement the algorithms and data structures that you intend to use, and in the last part you should use these for analyzing the stream.

## Reading the stream
The following code reads a remote file line by line. It is wrapped in a generator to make it easier to extend. You may modify this if you want to, but your solution should remain parametrized, so that your notebook can be run without having to consume the entire file.

In [1]:
import urllib

def stream(n):
    i = 0
    with urllib.request.urlopen('https://files.dtu.dk/fss/public/link/public/stream/read/traffic_2?linkToken=_DcyO-U3MjjuNzI-&itemName=traffic_2') as f:
        for line in f:
            element = line.rstrip().decode("utf-8")
            yield element
            i += 1
            if i == n:
                break

In [2]:
STREAM_SIZE = 1000
web_traffic_stream = stream(STREAM_SIZE)

#for e in web_traffic_stream:
    #print(e)

## Data structures

In [3]:
import mmh3
import math


class HyperLogLog: 
    def __init__(self):
        self._part = 10
        self._m = 2**self._part
        self._alpha = 0.7213/(1+1.079/self._m)
        self._M = [0 for i in range(0,self._m)]
    
    
    def add(self, elem):
        x = mmh3.hash(str(elem), signed=False)
        binary = '{0:032b}'.format(x)
        #part = int(math.log(64, 2))
        upperpart = binary[:self._part]
        lowerpart = binary[self._part:]
        j = int(upperpart,2)

        self._M[j] = max(self._M[j], len(lowerpart) - (int(lowerpart,2).bit_length()+1))

        pass
    
    def hashElement(element):
        # mmh3 64 outputs a 128 bit hash split in to two 64 bit outputs. 
        #For this purpose returning one of them will be sufficient
        a,b = mmh3.hash64(str(element), signed=False)
        return a

    def estimate(self):
        E = self._alpha*self._m**2*(sum([2**-number for number in self._M])**-1)
        V = self._M.count(0)
        
        if E <= (5/2)*self._m:
            if V != 0:
                EStar = self.LinearCounting(V)*2
            else:
                EStar = E
        elif E <= (1/30)*pow(2,32):
            EStar = E
        else: 
            EStar = -pow(2^32)*math.log2(1-E/pow(2,32))
        
        return EStar
    
    def LinearCounting(self, V):
        return self._m*math.log2(self._m/V)
    
    def Merge(self, other):
        res = HyperLogLog()
        res._M = [max(self._M[i],other._M[i]) for i in range(0,self._m)]
        return res

In [4]:
import numpy as np

## Count min sketch

d = 64
w = 64
def createM(d, w): 
    table = np.zeros([d, w])
    return table

def generateSeeds(d, w):
    seeds = np.random.randint(w, size = d)
    return seeds


def increment(seeds, key, table, d, w):
    for i in range(0, d):
        index = mmh3.hash(key, seeds[i]) % w
        table[i, index] = table[i, index]+1 
    return table

M = createM(d, w)
seeds = generateSeeds(d,w)
web_traffic_stream = stream(STREAM_SIZE)
for e in web_traffic_stream:
    increment(seeds,e,M,d,w)
    
def estimateCountMinResult(seeds, table, key, d, w):
        min_est = 100000
        for i in range(0, d):
            index = mmh3.hash(key, seeds[i]) % w
            #print("index" + str(index))
            #print("value" + str(table[i, index]))
            if table[i, index] < min_est:
                min_est = table[i, index]
        return min_est

def init_countMinSketch(d,w,seed):
    M = createM(d, w)
    return M

def addElementToCountMinSketch(element,M):
    M = increment(seed,ip,M,d,w)
    return M
    
def init_countMinSketchIp(d, w, seed):
    web_traffic_stream = stream(STREAM_SIZE)
    M = createM(d, w)
    
    for e in web_traffic_stream:
        split = e.split()
        ip = split[0]
        domain = split[1]
        M = increment(seed,ip,M,d,w)
    return M

def init_countMinSketchDomain(d, w, seed):
    web_traffic_stream = stream(STREAM_SIZE)
    M = createM(d, w)

    for e in web_traffic_stream:
        split = e.split()
        ip = split[0]
        domain = split[1]
        increment(seed,domain,M,d,w)
    return M

def countIp_countMinSketch(seed, M, targetIp, d, w):
    return (estimateCountMinResult(seed, M, targetIp, d,w))

def countDomain_countMinSketch(seed, M, targetDomain, d, w):
    return (estimateCountMinResult(seed, M, targetDomain, d,w))

def countAllDomain(seed, M, targetDomain, d, w):
    web_traffic_stream = stream(STREAM_SIZE)
    for e in web_traffic_stream:
        print(estimateCountMinResult(seed, M, targetDomain, d,w))
        
def splitElement(e):
    split = e.split()
    ip = split[0]
    domain = split[1]
    return ip, domain

### Queries 
 To answer the different queries we use the datastructures in a number of different ways. 
 We start by answering _How many unique IPs are there in the stream?_ by initializing a HLL(HyperLogLog) datastructure(ds) with IP addresse.
 
 Then we answer _How many unique IPs are there for each domain?_ by having a dictionary with incomming domains as keys and a HLL ds as value. This could potentially take up a lot of space. We assume here that the domains are limited. If we were to do this without the assumption we could keep a hashmap, hash the incomming domain and use the hash as index. Then we would keep the space limited to the size of the hashmap, but we would get a greater error. 
 
 To answer _How many times was IP X seen on domain Y? (for some X and Y provided at run time)_ we initalize a countMin Sketch on the whole stream with the whole element _e_ as input. As the element _e_ is composed of an ip address X and a domain Y, we can then just query the CountMinDataStructure as X concatenated with Y as the elements in the stream are composed. 
 
 To answer the question _How many unique IPs are there for the domains $d_1, d_2, \ldots$?_ we use the HLLs from the second question and use the merge function on the HLL. 

## Doing the different queries
Preparing the data structures:

In [5]:
from collections import defaultdict 

HLL_IPs = HyperLogLog()
seed = generateSeeds(d,w)
countMin = init_countMinSketch(64,64, seed)


web_traffic_stream = stream(STREAM_SIZE)
domains = {}

for e in web_traffic_stream:
    ip, domain = splitElement(e)
    HLL_IPs.add(ip)
    countMin = addElementToCountMinSketch(ip,countMin)
    if domain in domains:
        domains[domain].add(ip)
    else:
        dHLL = HyperLogLog()
        dHLL.add(ip)
        domains[domain] = dHLL
    

_How many unique IPs are there for each domain?_

In [6]:
HLL_IPs.estimate()

678.1235522227261

How many unique IPs are there for each domain?

In [7]:
for domain in domains: 
    print(domain, "          " ,domains[domain].estimate())

python.org            190.68806019375404
wikipedia.org            348.72783854375194
pandas.pydata.org            93.80595632767948
dtu.dk            2.8867998815669296
google.com            17.363259236975825
databricks.com            8.668875002262649
github.com            8.668875002262649
spark.apache.org            5.776423039751338
datarobot.com            2.8867998815669296


How many times was IP X seen on domain Y? 

In [8]:
key = '138.74.228.219'
print(estimateCountMinResult(seed, M, key, 64, 64))

key2 = '202.152.82.171'
print(estimateCountMinResult(seed, M, key2, 64, 64))

8.0
8.0


_How many unique IPs are there for the domains $d_1, d_2, \ldots$?_

In [9]:
domain1 = 'pandas.pydata.org'
domain2 = 'wikipedia.org'
domain3 = 'google.com'
domain4 = 'datarobot.com'

mergeHLL1_2 = domains[domain1].Merge(domains[domain2])
print('Merge between', domain1, 'and', domain2, 'has' , mergeHLL1_2.estimate(), 'unique IPs')

mergeHLL3_4 = domains[domain3].Merge(domains[domain4])
print('Merge between', domain3, 'and', domain4, 'has' , mergeHLL3_4.estimate(), 'unique IPs')

mergeHLL1234 = mergeHLL1_2.Merge(mergeHLL3_4)
print('Merge between', domain1, ',', domain2 , ',', domain3,'and', domain4, 'has' , mergeHLL1234.estimate(), 'unique IPs')


Merge between pandas.pydata.org and wikipedia.org has 437.71980257594186 unique IPs
Merge between google.com and datarobot.com has 20.26708201984003 unique IPs
Merge between pandas.pydata.org , wikipedia.org , google.com and datarobot.com has 461.23610193679673 unique IPs


## Analysis